<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/TCE_updated_code_240630.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEC application (updataed) as of 2024. 06. 30

## [1] Original code

In [ ]:
%%capture
!pip install gradio
!pip install pandas

In [ ]:
import gradio as gr
import pandas as pd

# Load the DataFrame
url = "TExam_new.csv"
df = pd.read_csv(url, encoding='utf-8-sig')

# Function to search years based on the selected mode
def search_years(search_by_year, search_by_keywords, query):
    if search_by_year and not search_by_keywords:
        # Search by matching the first four characters of the 'YEAR' column
        matches = df[df['YEAR'].str.startswith(query[:4])]
        if matches.empty:
            return "No results found for your query."
        return ", ".join(matches['YEAR'].tolist())  # Return a string of matched years
    elif search_by_keywords and not search_by_year:
        # Original keyword search logic
        keyword_list = [keyword.strip() for keyword in query.split(',')]
        matches = df[df['KEYWORDS'].apply(lambda x: any(keyword in x for keyword in keyword_list))]
        if matches.empty:
            return "No results found for your query."
        return ", ".join(matches['YEAR'].tolist())  # Return a string of matched years
    else:
        return "Please select exactly one search mode."


# Modified function to return keywords above the HTML image tag
# ...

def get_image_html(year):
    match = df[df['YEAR'] == year]
    if not match.empty:
        # Assuming 'LINK' column has the partial path like '2006_1.PNG'
        image_filename = match.iloc[0]['Filename']
        # Construct the full URL by appending the filename to the base path
        image_url = f'https://huggingface.co/spaces/MK-316/TCE/raw/main/TExams/{image_filename}'
        keywords = match.iloc[0]['KEYWORDS']
        # Place keywords above the image
        return f"<b>🌷 Keywords:</b> 🔑 {keywords}<br><img src='{image_url}' width='800'/>"
    else:
        return "No keywords found for this year.", "No image found for this year."

# ...


# Create Gradio Blocks interface
with gr.Blocks() as app:
    gr.Markdown("# Teacher Certificate Exam Searching Engine")
    gr.Markdown("## ❄️ [1] Search Data")

    # Row for checkboxes
    with gr.Row():
        search_by_year = gr.Checkbox(label="Search by YEAR", value=False)
        search_by_keywords = gr.Checkbox(label="Search Years by Keywords", value=False)

    # Row for search query, button and output
    with gr.Row():
        search_input = gr.Textbox(label="Search Query: e.g., 2024 (by YEAR) or tapping (by Keywords)",
                                  placeholder="Enter year or keywords separated by commas")
        search_button = gr.Button("Click to Search")
        search_output = gr.Text(label="Results (file names)")

    # Define actions
    search_button.click(fn=search_years, inputs=[search_by_year, search_by_keywords, search_input], outputs=search_output)

    gr.Markdown("## ❄️ [2] Display a Specific Exam Question Requested")
    with gr.Row():
        year_input = gr.Textbox(label="Type a file name among the result items: e.g., Year_item_part(a,b)", placeholder="Enter Year like '2024_01'")
        submit_button = gr.Button("Show me the exam question")  # Updated button text
    image_output = gr.HTML()

    # Define action for the submit button
    submit_button.click(fn=get_image_html, inputs=year_input, outputs=image_output)  # Use the button to trigger the display function

# Launch the app with sharing options
app.launch(debug=True)


# Revised version using dropdown box

# [2] New TCE app.py

In [ ]:
import gradio as gr
import pandas as pd

# Load the DataFrame
url = "TExam_new.csv"
df = pd.read_csv(url, encoding='utf-8-sig')

# Function to search years based on the selected mode
def search_years(search_by_year, search_by_keywords, query):
    if search_by_year and not search_by_keywords:
        # Search by matching the first four characters of the 'YEAR' column
        matches = df[df['YEAR'].str.startswith(query[:4])]
        if matches.empty:
            return [], "No results found for your query."
        return matches['YEAR'].tolist(), "Search completed successfully."
    elif search_by_keywords and not search_by_year:
        # Original keyword search logic
        keyword_list = [keyword.strip() for keyword in query.split(',')]
        matches = df[df['KEYWORDS'].apply(lambda x: any(keyword in x for keyword in keyword_list))]
        if matches.empty:
            return [], "No results found for your query."
        return matches['YEAR'].tolist(), "Search completed successfully."
    else:
        return [], "Please select exactly one search mode."

# Function to get image HTML
def get_image_html(year):
    match = df[df['YEAR'] == year]
    if not match.empty:
        # Assuming 'LINK' column has the partial path like '2006_1.PNG'
        image_filename = match.iloc[0]['Filename']
        # Construct the full URL by appending the filename to the base path
        image_url = f'https://huggingface.co/spaces/MK-316/TCE/raw/main/TExams/{image_filename}'
        keywords = match.iloc[0]['KEYWORDS']
        # Place keywords above the image
        return f"<b>🌷 Keywords:</b> 🔑 {keywords}<br><img src='{image_url}' width='800'/>"
    else:
        return "No keywords found for this year.", "No image found for this year."

# Create Gradio Blocks interface
with gr.Blocks() as app:
    gr.Markdown("# Teacher Certificate Exam Searching Engine")
    gr.Markdown("## ❄️ [1] Search Data")

    # Row for checkboxes
    with gr.Row():
        search_by_year = gr.Checkbox(label="Search by YEAR", value=False)
        search_by_keywords = gr.Checkbox(label="Search Years by Keywords", value=False)

    # Row for search query, button and output
    with gr.Row():
        search_input = gr.Textbox(label="Search Query: e.g., 2024 (by YEAR) or tapping (by Keywords)",
                                  placeholder="Enter year or keywords separated by commas")
        search_button = gr.Button("Click to Search")
        search_output = gr.Dropdown(label="Results (file names)", choices=[], visible=False)  # Hidden dropdown for results
        status_output = gr.Textbox(label="Status", visible=False)  # Hidden status textbox

    # Define actions
    def update_dropdown(results, status_message):
        return gr.update(choices=results), status_message

    search_button.click(fn=search_years, inputs=[search_by_year, search_by_keywords, search_input],
                        outputs=[search_output, status_output],
                        postprocess=update_dropdown)

    gr.Markdown("## ❄️ [2] Display a Specific Exam Question Requested")
    with gr.Row():
        year_input = gr.Dropdown(label="Select a file name from the results", choices=[])
        submit_button = gr.Button("Show me the exam question")
    image_output = gr.HTML()

    # Update the year input dropdown based on the search results
    def update_year_input(results):
        return gr.update(choices=results)

    search_output.change(fn=update_year_input, inputs=search_output, outputs=year_input)

    # Define action for the submit button
    submit_button.click(fn=get_image_html, inputs=year_input, outputs=image_output)

# Launch the app with sharing options
app.launch(debug=True)


## [3] Search option to update (Final code) 0630

In [ ]:
import gradio as gr
import pandas as pd

# Load the DataFrame
url = "TExam_new.csv"
df = pd.read_csv(url, encoding='utf-8-sig')

# Function to search years based on the selected mode
def search_years(search_mode, query):
    if search_mode == "Search by YEAR":
        matches = df[df['YEAR'].str.startswith(query[:4])]
    elif search_mode == "Search Years by Keywords":
        keyword_list = [keyword.strip() for keyword in query.split(',')]
        matches = df[df['KEYWORDS'].apply(lambda x: any(keyword in x for keyword in keyword_list))]
    else:
        return [], "Please select exactly one search mode."

    if matches.empty:
        return [], "No results found for your query."
    return matches['YEAR'].tolist(), "Search completed successfully."

# Function to get image HTML
def get_image_html(year):
    match = df[df['YEAR'] == year]
    if not match.empty:
        image_filename = match.iloc[0]['Filename']
        image_url = f'https://huggingface.co/spaces/MK-316/TCE/raw/main/TExams/{image_filename}'
        keywords = match.iloc[0]['KEYWORDS']
        return f"<b>🌷 Keywords:</b> 🔑 {keywords}<br><img src='{image_url}' width='800'/>"
    else:
        return "No keywords found for this year.", "No image found for this year."

# Create Gradio Blocks interface
with gr.Blocks() as app:
    gr.Markdown("# Teacher Certificate Exam Searching Engine")
    gr.Markdown("## ❄️ [1] Search Data")

    # Radio buttons to select search mode
    search_mode = gr.Radio(choices=["Search by YEAR", "Search Years by Keywords"], label="Search Mode")

    # Row for search query and button
    search_input = gr.Textbox(label="Search Query: e.g., 2024 (by YEAR) or tapping (by Keywords)", placeholder="Enter year or keywords")
    search_button = gr.Button("Click to Search")
    search_output = gr.Dropdown(label="Results (file names)", choices=[], visible=False)
    status_output = gr.Textbox(label="Status", visible=False)

    # Connect actions
    search_button.click(fn=search_years, inputs=[search_mode, search_input], outputs=[search_output, status_output])

    gr.Markdown("## ❄️ [2] Display a Specific Exam Question Requested")
    year_input = gr.Dropdown(label="Select a file name from the results", choices=[])
    submit_button = gr.Button("Show me the exam question")
    image_output = gr.HTML()

    # Update dropdown and image display
    search_output.change(fn=lambda results: gr.update(choices=results), inputs=search_output, outputs=year_input)
    submit_button.click(fn=get_image_html, inputs=year_input, outputs=image_output)

# Launch the app with sharing options
app.launch(debug=True)
